In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '../terrace')

%load_ext autoreload
%autoreload 2

In [20]:
from glob import glob
import subprocess

lit_pcba_folder = "/home/boris/Data/LIT-PCBA"
dist = 5
for target in glob(lit_pcba_folder + "/*"):
    prot_files = glob(target + "/*_protein.mol2")
    pdb_ids = [ f.split("/")[-1].split("_")[0] for f in prot_files ]
    out_file = target + "/process.pml"
    print(f"Writing to {out_file}")
    with open(out_file, "w") as f:
        for pdb in pdb_ids:
            f.write(f"load {pdb}_protein.mol2\n")
            f.write(f"load {pdb}_ligand.mol2\n")
            f.write(f"create {pdb}, ({pdb}_protein or {pdb}_ligand)\n")
            f.write(f"remove {pdb}_protein\n")
            f.write(f"remove {pdb}_ligand\n")
        f.write("""
            remove solvent
            remove inorganic
            """)
        to_align = pdb_ids[0]
        for pdb in pdb_ids[1:]:
            f.write(f"align {pdb}, {to_align}\n")
        for pdb in pdb_ids:
            poc = pdb + "_pocket"
            out_pdb = target + f"/{poc}.pdb"
            f.write(f"select byres (({pdb} within {dist} of organic) and not organic)\n")
            f.write(f"set_name sele, {poc}\n")
            f.write(f"save {out_pdb}, {poc}\n")

Writing to /home/boris/Data/LIT-PCBA/GBA/process.pml
Writing to /home/boris/Data/LIT-PCBA/ALDH1/process.pml
Writing to /home/boris/Data/LIT-PCBA/ADRB2/process.pml
Writing to /home/boris/Data/LIT-PCBA/ESR1_ant/process.pml
Writing to /home/boris/Data/LIT-PCBA/ESR1_ago/process.pml
Writing to /home/boris/Data/LIT-PCBA/FEN1/process.pml
Writing to /home/boris/Data/LIT-PCBA/KAT2A/process.pml
Writing to /home/boris/Data/LIT-PCBA/PKM2/process.pml
Writing to /home/boris/Data/LIT-PCBA/MTORC1/process.pml
Writing to /home/boris/Data/LIT-PCBA/IDH1/process.pml
Writing to /home/boris/Data/LIT-PCBA/VDR/process.pml
Writing to /home/boris/Data/LIT-PCBA/PPARG/process.pml
Writing to /home/boris/Data/LIT-PCBA/MAPK1/process.pml
Writing to /home/boris/Data/LIT-PCBA/OPRK1/process.pml
Writing to /home/boris/Data/LIT-PCBA/TP53/process.pml


In [48]:
from datasets.lit_pcba import LitPcbaDataset
from common.cfg_utils import *

cfg = get_config("../configs", "short_thicc_op_gnn")
print(LitPcbaDataset.get_all_targets(cfg))
dataset = LitPcbaDataset(cfg, "VDR")
dataset[0].is_active

['GBA', 'ALDH1', 'process.sh', 'ADRB2', 'ESR1_ant', 'ESR1_ago', 'FEN1', 'KAT2A', 'PKM2', 'MTORC1', 'IDH1', 'VDR', 'PPARG', 'MAPK1', 'OPRK1', 'TP53']


tensor(True)

In [49]:
from datasets.make_dataset import *
bb_all = make_dataset(cfg, "all")

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [54]:
import wandb
from lit_pcba_screen import *

run_id = "1nhqz8vw"
api = wandb.Api()
run = api.run(f"{cfg.project}/{run_id}")
cfg = get_run_config(run, cfg)
preds = get_lit_pcba_preds(cfg, run, "VDR")

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [73]:
import torch
scores = torch.cat(preds).to('cpu')
dataset = LitPcbaDataset(cfg, "VDR")
yt = dataset.get_all_yt()

In [78]:
mets = get_screen_metrics(scores, yt)

In [81]:
target = "VDR"

In [84]:
row = mets
row["target"] = "VDR"
rows = []
rows.append(row)
import pandas as pd
pd.DataFrame(rows)

,EF1%,NEF1%,total chosen,total actives chosen,total actives in set,target
0,tensor(3.0541),tensor(0.0305),3563,tensor(27),tensor(884),VDR


In [117]:
bb_all.activities.pocket[bb_all.activities.pocket.str.contains("ALDH")].unique()

array([], dtype=object)

In [108]:
bb_val = make_dataset(cfg, "val")
bb_test = make_dataset(cfg, "test")

In [121]:
LitPcbaDataset.get_all_targets(cfg)

['GBA',
 'ALDH1',
 'ADRB2',
 'ESR1_ant',
 'ESR1_ago',
 'FEN1',
 'KAT2A',
 'PKM2',
 'MTORC1',
 'IDH1',
 'VDR',
 'PPARG',
 'MAPK1',
 'OPRK1',
 'TP53']

In [113]:
bb_train.activities.query("pocket == 'PPARG_HUMAN_229_505_0'")

,Unnamed: 0,lig_smiles,lig_file,uniprot,pocket,ex_rec_file,ex_rec_pdb,ex_rec_pocket_file,num_pocket_residues,pocket_center_x,pocket_center_y,pocket_center_z,pocket_size_x,pocket_size_y,pocket_size_z,active
34348,17321,CC(C)(Oc1ccc(CCCN(CCc2c(F)cccc2Cl)C(=O)Nc2cccc...,chembl_structures/mol_21529.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/1k74_D_rec.pdb,1k74,PPARG_HUMAN_229_505_0/1k74_D_rec_pocket.pdb,61,-24.094501,-19.836,7.894,24.247,33.432,30.410001,True
34349,17321,CCOc1ccc(-c2c(Cl)ncn2-c2ccc(S(C)(=O)=O)cc2)cc1F,chembl_structures/mol_35899.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/1k74_D_rec.pdb,1k74,PPARG_HUMAN_229_505_0/1k74_D_rec_pocket.pdb,61,-24.094501,-19.836,7.894,24.247,33.432,30.410001,False
55636,28094,Cc1nc(-c2ccc(C(F)(F)F)cc2)oc1CSc1ccc(OCC(=O)O)cc1,chembl_structures/mol_30266.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/4ci5_A_rec.pdb,4ci5,PPARG_HUMAN_229_505_0/4ci5_A_rec_pocket.pdb,53,-24.094501,-19.836,7.894,24.247,33.432,30.410001,True
55637,28094,CCCCCCCCCCOC(=O)NC1CCS(=O)(=O)C1,chembl_structures/mol_129959.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/4ci5_A_rec.pdb,4ci5,PPARG_HUMAN_229_505_0/4ci5_A_rec_pocket.pdb,53,-24.094501,-19.836,7.894,24.247,33.432,30.410001,False
55638,28095,Cc1cc(SCc2oc(-c3ccc(C(F)(F)F)cc3)nc2C)ccc1OCC(...,chembl_structures/mol_30267.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/5tto_A_rec.pdb,5tto,PPARG_HUMAN_229_505_0/5tto_A_rec_pocket.pdb,55,-24.094501,-19.836,7.894,24.247,33.432,30.410001,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302093,672124,CC(C)NC1=NNC(=O)C1=Cc1cc2ccccc2[nH]1,chembl_structures/mol_331175.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/2g0g_A_rec.pdb,2g0g,PPARG_HUMAN_229_505_0/2g0g_A_rec_pocket.pdb,57,-24.094501,-19.836,7.894,24.247,33.432,30.410001,False
1302232,672194,O[C@@H]1CS[C@@H](Cn2cnc3c(NCc4cccc(I)c4)nc(Cl)...,chembl_structures/mol_608893.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/1zeo_A_rec.pdb,1zeo,PPARG_HUMAN_229_505_0/1zeo_A_rec_pocket.pdb,58,-24.094501,-19.836,7.894,24.247,33.432,30.410001,True
1302233,672194,NC(=O)CC[C@H](NC(=O)[C@H](CCC(=O)O)NC(=O)CCc1c...,chembl_structures/mol_421972.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/1zeo_A_rec.pdb,1zeo,PPARG_HUMAN_229_505_0/1zeo_A_rec_pocket.pdb,58,-24.094501,-19.836,7.894,24.247,33.432,30.410001,False
1302234,672195,O[C@@H]1CO[C@@H](Cn2cnc3c(NCc4cccc(I)c4)nc(Cl)...,chembl_structures/mol_608894.sdf,P37231,PPARG_HUMAN_229_505_0,PPARG_HUMAN_229_505_0/4jl4_A_rec.pdb,4jl4,PPARG_HUMAN_229_505_0/4jl4_A_rec_pocket.pdb,60,-24.094501,-19.836,7.894,24.247,33.432,30.410001,True
